#**Concept Edges**

In [11]:
import requests
import re
url_concept_net = 'http://api.conceptnet.io' 

In [12]:
def clear_name(name):
  if "_" in name:
    name = name.replace("_", " ")
  name = name.lower()

  if bool(re.search("the ", name)):
    name = re.sub(r"the ", "", name)

  if bool(re.search("a ", name)):
    name = re.sub(r"a ", "", name)

  if bool(re.search("an ", name)):
    name = re.sub(r"an ", "", name)

  if bool(re.search("my ", name)):
    name = re.sub(r"my ", "", name)

  if bool(re.search("is ", name)):
    name = re.sub(r"is ", "", name)

  if bool(re.search("is$", name)):
    name = re.sub(r"is$", "", name)
   
  if bool(re.search("your ", name)):
    name = re.sub(r"your ", "", name)
  
  if bool(re.search("any ", name)):
    name = re.sub(r"any ", "", name)

  if bool(re.search("every ", name)):
    name = re.sub(r"every ", "", name)

  if bool(re.search("own ", name)):
    name = re.sub(r"own ", "", name)

  if bool(re.search("to ", name)):
    name = re.sub(r"to ", "", name)

  if bool(re.search("something ", name)):
    name = re.sub(r"something ", "", name)
  
  name = name.strip()
  return name

In [13]:
def make_recursive_call(results, query):
   response = requests.get(query)
   result = response.json()
   results.append(result)
   if "view" in result.keys():
    view = result["view"]
    #print(view.keys())
    if "nextPage" in view.keys():
      next_page = view["nextPage"]
      new_query = url_concept_net + next_page
      make_recursive_call(results, new_query)


In [14]:
def make_call(label, node):
  results = []
  request_string = url_concept_net + "/query?"+node+"=/c/en/"+label+"&limit=500"
  response = requests.get(request_string)
  result = response.json()
  #print(result)
  results.append(result)
  if "view" in result.keys():
    view = result["view"]
    #print(view.keys())
    if "nextPage" in view.keys():
      next_page = view["nextPage"]
      new_query = url_concept_net + next_page
      make_recursive_call(results, new_query)
  return results

In [15]:
def get_nodes(conceptNet_results, starting_word):
  nodes = []
  count = 1
  for result in conceptNet_results:
    edges = result["edges"]
    for edge in edges:
      rel_type = edge["rel"]["label"]
      end_node = clear_name(edge["end"]["label"])
      start_node = clear_name(edge["start"]["label"])
      if starting_word == start_node:
        label_name = clear_name(end_node)
        pair = {"label":label_name,
                "rel_type":rel_type}
        nodes.append(pair)

      elif starting_word == end_node:
        label_name = clear_name(start_node)
        pair = {"label":label_name,
              "rel_type":rel_type}
        nodes.append(pair)
    count = count + 1
  return nodes

In [16]:
def update_results_map(results_map, word_one, word_two, edges_count_map):
  words = []
  words.append(word_one)
  words.append(word_two)
  words = sorted(words)
  key = words[0] + "_" + words[1]
  if not key in results_map.keys():
    results_map[key] = {}

  for rel_type in edges_count_map.keys():
    current_count = edges_count_map[rel_type]
    if rel_type in results_map[key]:
      old_count = results_map[key][rel_type]
      current_count = current_count + old_count
    results_map[key][rel_type] = current_count



In [17]:
def compute_totals(results_map):
  for key, locations in results_map.items():
    total = 0
    for location in locations:
      current_count = results_map[key][location]
      total = total + current_count
    results_map[key]["total"] = total


In [18]:
def print_map(results_map):
  for key, relations in results_map.items():
    tokens = key.split("_")
    word_one = tokens[0].strip()
    word_two = tokens[1].strip()
    print(word_one, "-", word_two)
    for relation in relations:
      if relation == "total":
        continue
      count = results_map[key][relation]
      print(" -", relation, "->", count)
    total_count = results_map[key]["total"]
    print(" => Total:", total_count)
    print()

In [19]:
words = ["book", "library", "person"]
results_map = {}
#relations = ["LocatedNear", "AtLocation"]
for i in range(len(words)):
  edges_count_map = {}
  word_one = words[i]
  conceptNet_results = make_call(word_one, "start")
  nodes = get_nodes(conceptNet_results, word_one)
  #print("For", word_one, "found", len(nodes))
  for j in range(len(words)):
    word_two = words[j]
    if word_one == word_two:
      continue
    for node in nodes:
      node_name = node["label"]
      node_rel = node["rel_type"]
      if node_name == word_two:
        if not node_rel in edges_count_map.keys():
          edges_count_map[node_rel] = 0
        old_count = edges_count_map[node_rel] 
        new_count = old_count + 1
        edges_count_map[node_rel] = new_count

    update_results_map(results_map, word_one, word_two, edges_count_map)

compute_totals(results_map)
print_map(results_map)

book - library
 - PartOf -> 1
 - RelatedTo -> 2
 - AtLocation -> 1
 - LocatedNear -> 1
 - Antonym -> 1
 => Total: 6

book - person
 - PartOf -> 1
 - RelatedTo -> 1
 - AtLocation -> 1
 - LocatedNear -> 1
 => Total: 4

library - person
 - RelatedTo -> 1
 - Antonym -> 1
 => Total: 2

